In [ ]:
import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
from cvzone.PlotModule import LivePlot
import time

In [ ]:
cap = cv2.VideoCapture(0)

detector = FaceMeshDetector()
ploty = LivePlot(540, 360, [10, 64])

idlist = [22, 23, 24, 26, 110, 157, 158, 159, 160, 161, 130, 243]
color = (0, 0, 255)

ratiolist = []
counter = 0
goz = 0
goz_kapali_baslangic = None
uyari_suresi = 2  # Uyarı için gözün kapalı kalması gereken süre (saniye)

In [ ]:
while True:
    ret, frame = cap.read()
    frame, faces = detector.findFaceMesh(frame, draw=False)

    if faces:
        face = faces[0]
        for id in idlist:
            cv2.circle(frame, face[id], 3, color, cv2.FILLED)

        leftup = face[159]
        leftdown = face[23]
        leftleft = face[130]
        leftright = face[243]

        length_ver, _ = detector.findDistance(leftup, leftdown)
        length_hor, _ = detector.findDistance(leftleft, leftright)

        cv2.line(frame, leftup, leftdown, (0, 255, 0), 3)
        cv2.line(frame, leftleft, leftright, (0, 255, 0), 3)

        ratio = int((length_ver / length_hor) * 100)
        ratiolist.append(ratio)

        if len(ratiolist) > 3:
            ratio = sum(ratiolist[-4:]) // 4

        if ratio < 35:
            if goz_kapali_baslangic is None:
                goz_kapali_baslangic = time.time()
            else:
                goz_kapali_sure = time.time() - goz_kapali_baslangic
                if goz_kapali_sure > uyari_suresi:
                    cvzone.putTextRect(frame, "Uyari: Goz uzun sure kapali!", (50, 150), scale=1.5, thickness=2, colorR=(0, 0, 255))
        else:
            if goz_kapali_baslangic is not None and time.time() - goz_kapali_baslangic > 0.3:
                goz += 1
            goz_kapali_baslangic = None
            color = (0, 0, 255)

        cvzone.putTextRect(frame, f"Blink Count: {goz}", (50, 100), colorR=color)

    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
